In [ ]:
import timm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Replace with your actual username and repo name
model = timm.models.create_model("hf_hub:Luna-Skywalker/convnext-tea-v2", pretrained=True)
model.eval()  # Set to evaluation mode
model.to(device)

config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/198M [00:00<?, ?B/s]

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)


In [ ]:
import json
from huggingface_hub import hf_hub_download

# Download the label file from the hub
labels_path = hf_hub_download(repo_id="Luna-Skywalker/convnext-tea-v2", filename="config.json")
with open(labels_path, 'r') as f:
    config = json.load(f)

class_names = config['label_names']
print("Labels:", class_names)

Labels: ['Anthracnose', 'algal leaf', 'bird eye spot', 'brown blight', 'gray light', 'healthy', 'red leaf spot', 'white spot']


In [ ]:
from torchvision import transforms
from PIL import Image

# Image preprocessing (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)  # Add batch dimension


In [ ]:
import torch.nn.functional as F

def predict(image_path):
    image_tensor = preprocess_image(image_path).to(device)
    with torch.no_grad():
        logits = model(image_tensor)
        probs = F.softmax(logits, dim=1).squeeze().tolist()

    result = {class_names[i]: round(probs[i], 4) for i in range(len(class_names))}
    return result


In [ ]:
# Provide a test image path
test_image_path = "/content/algal leaf.jpg"  # Replace with actual path
predictions = predict(test_image_path)
print("Predicted Probabilities:\n", predictions)

Predicted Probabilities:
 {'Anthracnose': 0.0, 'algal leaf': 0.9999, 'bird eye spot': 0.0, 'brown blight': 0.0, 'gray light': 0.0, 'healthy': 0.0, 'red leaf spot': 0.0, 'white spot': 0.0}


In [ ]:
# Provide a test image path
test_image_path1 = "/content/bird eye spot.jpg"  # Replace with actual path
predictions1 = predict(test_image_path1)
print("Predicted Probabilities:\n", predictions1)

Predicted Probabilities:
 {'Anthracnose': 0.0002, 'algal leaf': 0.0, 'bird eye spot': 0.9997, 'brown blight': 0.0, 'gray light': 0.0001, 'healthy': 0.0, 'red leaf spot': 0.0, 'white spot': 0.0}


In [ ]:
# Provide a test image path
test_image_path2 = "/content/images 1.jpg"  # Replace with actual path
predictions2 = predict(test_image_path2)
print("Predicted Probabilities:\n", predictions2)

Predicted Probabilities:
 {'Anthracnose': 0.0025, 'algal leaf': 0.374, 'bird eye spot': 0.0042, 'brown blight': 0.0592, 'gray light': 0.0001, 'healthy': 0.0096, 'red leaf spot': 0.2317, 'white spot': 0.3187}


In [ ]:
# Provide a test image path
test_image_path3 = "/content/images 2.jpg"  # Replace with actual path
predictions3 = predict(test_image_path3)
print("Predicted Probabilities:\n", predictions3)

Predicted Probabilities:
 {'Anthracnose': 0.7367, 'algal leaf': 0.1843, 'bird eye spot': 0.001, 'brown blight': 0.001, 'gray light': 0.0005, 'healthy': 0.0121, 'red leaf spot': 0.0107, 'white spot': 0.0538}
